In [1]:
import mitiq

mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [2]:
frontend = "cirq"

In [3]:
from mitiq import benchmarks

circuit = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type = frontend,
)[0]

print(circuit)

0: ───Y───X───X───Y^-0.5───Y^-0.5───


In [4]:
import numpy as np
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [4]:
from mitiq.zne import scaled_circuits, combine_results
from mitiq.zne.scaling import fold_gates_at_random

scale_factors = [2.0, 3.0]

folded_circuits = scaled_circuits(
    circuit=circuit,
    scale_factors=scale_factors,
    scale_method=fold_gates_at_random,
)

folded_circuits

[0: ───Y───X───X───Y^-0.5───Y^0.5───Y^-0.5───Y^-0.5───Y^0.5───Y^-0.5───,
 0: ───Y───Y───Y───X───X───X───X───X───X───Y^-0.5───Y^0.5───Y^-0.5───Y^-0.5───Y^0.5───Y^-0.5───]

In [6]:
results = [execute(circuit) for circuit in folded_circuits]

print(results)

[0.92561346, 0.8926793]


In [10]:
from typing import List
import cirq
import numpy as np
from mitiq.interface import mitiq_cirq
from mitiq import Executor, Observable, PauliString

def batch_compute_density_matrix(circuits: List[cirq.Circuit]) -> List[np.ndarray]:
    return [mitiq_cirq.compute_density_matrix(circuit) for circuit in circuits]


batched_executor = Executor(batch_compute_density_matrix, max_batch_size=10)

In [11]:
batched_executor.can_batch

True

In [12]:
# circuits = [cirq.Circuit(pauli.on(q)) for pauli in (cirq.X, cirq.Y, cirq.Z)]

results = batched_executor.evaluate(folded_circuits, Observable(PauliString("Z")))

print("Results:", results)
print("\nCalls to executor:", batched_executor.calls_to_executor)
print("\nExecuted circuits:\n", *batched_executor.executed_circuits, sep="\n")
print("\nQuantum results:\n", *batched_executor.quantum_results, sep="\n")

Results: [0.9615676999092102, 0.9434799551963806]

Calls to executor: 1

Executed circuits:

0: ───Y^-0.5───X^-0.5───Y^0.5───Y^-0.5───Y^0.5───Y^-0.5───I───I───I───(Z^-0.5-X^0.5)───(X^-0.5-Z^0.5)───(Z^-0.5-X^0.5)───
0: ───Y^-0.5───Y^0.5───Y^-0.5───X^-0.5───X^0.5───X^-0.5───Y^0.5───Y^-0.5───Y^0.5───Y^-0.5───Y^0.5───Y^-0.5───I───I───I───(Z^-0.5-X^0.5)───(X^-0.5-Z^0.5)───(Z^-0.5-X^0.5)───

Quantum results:

[[0.98078376+1.4752150e-08j 0.00473114+3.3582482e-02j]
 [0.00473114-3.3582367e-02j 0.01921606-1.4752150e-08j]]
[[0.9717397 -3.0100345e-08j 0.0138422 +4.2603709e-02j]
 [0.01384223-4.2603709e-02j 0.02825978-2.9504299e-08j]]


In [1]:
import functools
from unittest.mock import Mock
from mitiq.zne.inference import PolyFactory
from mitiq.zne import execute_with_zne
from mitiq.observable import Observable, PauliString
import cirq
import numpy as np

npX = np.array([[0, 1], [1, 0]])
"""Defines the sigma_x Pauli matrix in SU(2) algebra as a (2,2) `np.array`."""

npZ = np.array([[1, 0], [0, -1]])
"""Defines the sigma_z Pauli matrix in SU(2) algebra as a (2,2) `np.array`."""

# Default executor for unit tests
def executor(circuit) -> float:
    wavefunction = circuit.final_state_vector(
        ignore_terminal_measurements=True
    )
    return np.real(wavefunction.conj().T @ np.kron(npX, npZ) @ wavefunction)

def test_with_observable_batched_factory(executor):
    observable = Observable(PauliString(spec="Z"))
    circuit = cirq.Circuit(cirq.H.on(cirq.LineQubit(0))) * 20
    executor = functools.partial(
        executor, noise_model_function=cirq.depolarize
    )

    real_factory = PolyFactory(scale_factors=[1, 3, 5], order=2)
    mock_factory = Mock(spec_set=PolyFactory, wraps=real_factory)
    zne_val = execute_with_zne(
        circuit,
        executor=executor,
        observable=observable,
        factory=mock_factory,
    )

    mock_factory.run.assert_called_with(
        circuit, executor, observable, fold_gates_at_random, 1
    )
    assert 0 <= zne_val <= 2

In [2]:
from mitiq.interface.mitiq_cirq import (
    compute_density_matrix,
    sample_bitstrings,
)
test_with_observable_batched_factory((sample_bitstrings, compute_density_matrix))

TypeError: the first argument must be callable